Author: Carlos Roberto de Melo
    
Date: 12/13/2020
    
Goal: Jampy Emcee model for SDP.81.
Here we assume a dark matter component described by a pseudo NFW profile, a Gaussian ML ratio. Beside that, we let free the super massive black hole mass, the anisotropy parameter an the inclination.

The pseudo-NFW parameter has two free parameters: intensity (i.e $\rho_s$) and the axial ratio. While the gaussian ML has three parameters, the central mass-to-light ratio, the sigma gaussian (named delta here), and the lower value of the ML.

We assume flat prior for all parameters.

In [1]:
#Packeges

import numpy as np
from My_Jampy import JAM
import emcee
import matplotlib.pyplot as plt

from multiprocessing import Pool
import time
import os

from astropy.cosmology import Planck15 as cosmo

os.environ["OMP_NUM_THREADS"] = "1"


%config InlineBackend.figure_format = "retina"

from matplotlib import rcParams
rcParams["savefig.dpi"] = 100
rcParams["figure.dpi"] = 100
rcParams["font.size"] = 20

In [2]:
#Reading data
y_px, x_px, vrms, erms = np.loadtxt('pPXF_rot_data.txt', unpack=True)                  #pPXF
surf_star_dat, sigma_star_dat, qstar_dat = np.loadtxt('JAM_Input.txt', unpack=True)    #photometry
surf_DM_dat, sigma_DM_dat, qDM_dat  = np.loadtxt('pseudo-DM Input.txt', unpack=True)    #DM


muse_normpsf, muse_sigmapsf = np.loadtxt("MUSE_Psf_model.txt", unpack=True)

In [3]:
#--------------------------------------- EMCEE -----------------------------------------------------#


### Priors#--------------------------------------- EMCEE -----------------------------------------------------#


### Priors# parameter boundaries. [lower, upper]
boundary = {'inc': [70, 120], 'beta': [-5, 5], 'ml0': [0.5, 15], 'delta': [0.5, 2], 'lower': [0, 1],
            'log_mbh':[7, 11], 'qDM': [0.15, 1], 'log_rho_s':[6, 13] }


def gaussian_ml(sigma, delta, ml0=1.0, lower=0.0):
    '''
    Create a M*L gradient
    sigma: Gaussian sigma [arcsec]
    delta: Gradient value
    ml0: Central stellar mass to light ratio
    lower: the ratio between the central and the outer most M*/L
    '''

    sigma = np.atleast_1d(sigma)
    sigma = sigma - sigma[0]
    ML = ml0 * (lower + (1-lower)*np.exp(-0.5 * (sigma * delta)**2))
    
    return ML


def check_Deprojected_axial(parsDic):
    inc = np.radians(parsDic['inc'])
    #Stellar
    qintr_star = qstar_dat**2 - np.cos(inc)**2
    if np.any(qintr_star <= 0):
        return -np.inf
    
    qintr_star = np.sqrt(qintr_star)/np.sin(inc)
    if np.any(qintr_star <= 0.05):
        return -np.inf
    
        #DM
    qintr_DM = parsDic['qDM']**2 - np.cos(inc)**2
    if qintr_DM <= 0:
        return -np.inf
    
    qintr_DM = np.sqrt(qintr_DM)/np.sin(inc)
    if qintr_DM <= 0.05:
        return -np.inf

    return 0.0

    


def check_boundary(parsDic):
    """
        Check whether parameters are within the boundary limits
        input
            parsDic: parameter dictionary {'paraName', value}
        output
            -np.inf or 0.0
    """   
    
    
    #Check if beta is ok

    #Avoid beta[i] == 1, because this could cause problems
    if any(parsDic['beta'] == 1):
        return -np.inf
    else:
        pass

    #Check beta boundary
    for i in range(len(parsDic['beta'])):
        if boundary['beta'][0] < parsDic['beta'][i] < boundary['beta'][1] :
            pass
        else:
            return -np.inf


    #Check if deprojected axial ratio is ok  (q' <=0 or q' <= 0.05) for the dynamical model.
    if not np.isfinite(check_Deprojected_axial(parsDic)):
        return -np.inf

    #Check if the others parameters are within the boundary limits
    keys = set(parsDic.keys())
    excludes = set(['beta'])  #Exclude beta and ml, because we already verify above


    for keys in keys.difference(excludes):
        if boundary[keys][0] < parsDic[keys] < boundary[keys][1]:
            pass
        else:
            return -np.inf
    return 0.0

def log_prior(parsDic):
    '''
    Calculate the prior lnprob
    input
      parsDic: parameter dictionary {'paraName', value}
    output
      lnprob
    '''
    
    rst = 0
    """
    The lines above is only for doble check, because we are assuming flat prior for all parameters Once we got here, all values ​​have already been accepted, so just return 0.0 for one of them
    """

    rst += 0.0     #ml0
    rst += 0.0     #beta
    rst += 0.0     #inc
    rst += 0.0     #log_mbh
    rst += 0.0     #qDM
    rst += 0.0     #log_rho_s
    rst += 0.0     #delta
    rst += 0.0     #lower
    
    return rst


def Updt_JAM(parsDic):
    '''
       Update the dynamical mass model
    input
      parsDic: parameter dictionary {'paraName', value}
    '''
    surf_DM_model = surf_DM_dat*(10**parsDic['log_rho_s'])       #DM mass surface density
    qDM_model = np.ones(qDM_dat.shape)*parsDic['qDM']            #DM axial ratio
    beta_model = np.array(parsDic['beta'])                       #anisotropy parameter
    mbh_model = 10**parsDic['log_mbh']                           #BH mass
    
    #mass-to-light update 
    ml_model = gaussian_ml(sigma=sigma_star_dat, delta=parsDic['delta'],
                           ml0=parsDic['ml0'], lower=parsDic['lower'])
    


    #Model Updt
    Jampy_model.upt(surf_dm=surf_DM_model, qobs_dm=qDM_model, inc=parsDic['inc'],
                     ml=ml_model, beta=parsDic['beta'], mbh=mbh_model)
    
def JAM_log_likelihood(parsDic):
    """
        Perform JAM modeling and return the chi2
    """
    
    Updt_JAM(parsDic)               #Updt values for each iteration
    
    rmsModel, ml, chi2, chi2T = Jampy_model.run()
    return -0.5 * chi2T

def log_probability(pars):
    """
        Log-probability function for whole model.
        input:
            pars: current values in the Emcee sample.
        output:
            log probability for the combined model.
    """

    (ml0, delta, lower, b1, b2, b3, b4, b5, b6, b7, b8, inc, log_mbh, qDM, log_rho_s) = pars
    

    
    beta =  np.array([b1, b2, b3, b4, b5, b6, b7, b8])
    parsDic = {'ml0': ml0, 'delta': delta, 'lower':lower, 'inc': inc, 'qDM': qDM, 'log_rho_s': log_rho_s,
               'log_mbh': log_mbh, 'beta': beta}
    
    #Checking boundaries
    if not np.isfinite(check_boundary(parsDic)):
        return -np.inf
    #calculating the log_priors
    lp = log_prior(parsDic)

    return lp + JAM_log_likelihood(parsDic)

In [4]:
#Initializing the model.

#Galaxy/telescope intrinsic parameters
distance = cosmo.angular_diameter_distance(0.299).value     #Angular diameter distance [Mpc]
pixsize=0.2                                                 #pixscale of IFU [arcsec/px]
inc = 85                                                    #Inclination [deg]
log_mbh = 8                                                 #Log Mass of black hole [log(M_sun)]
beta0 = np.ones_like(surf_star_dat)*0.3                     #Anisotropy parameter, one for each gaussian component
log_rho_s = 8                                               #Log Dark Matter intensity component [log(M_sun/pc²)]
ML0 = gaussian_ml(sigma=sigma_star_dat, 
                      delta=0.5, ml0=10, lower=0.4)         #Gaussian Mass-to-light ratio [M_sun/L_sun]

#Create the model
Jampy_model = JAM(ybin=y_px, xbin=x_px,inc=inc, distance=distance, mbh= 10**log_mbh,
                  rms=vrms, erms=erms, beta=beta0, normpsf=muse_normpsf, sigmapsf=muse_sigmapsf, pixsize=pixsize)

#Add Luminosity component
Jampy_model.luminosity_component(surf_lum=surf_star_dat, sigma_lum=sigma_star_dat,
                                    qobs_lum=qstar_dat, ml=ML0)

#Add Dark Matter component
Jampy_model.DM_component(surf_dm=(10**log_rho_s)*surf_DM_dat, sigma_dm=sigma_DM_dat, qobs_dm=qDM_dat)

In [5]:
# Realistic anisotropy profile from a Schwarzschild model.
# The anisotropy varies smoothly between the following three regimes:
# 1. beta = -1 for R < 1"
# 2. beta = 0.3 for 1" < R < 30"
# 3. beta = -0.2 for R > 30"
#
beta0 = np.empty_like(sigma_star_dat)
beta0[sigma_star_dat <= 1] = -1.0
beta0[(sigma_star_dat > 1) & (sigma_star_dat <= 30)] = 0.3
beta0[sigma_star_dat > 30] = -0.2

In [6]:
#------------------------EMCEE--------------------------#

#Initial positions
"""
    Pay close attention to the order in which the components are added. 
    They must follow the log_probability unpacking order.
"""

ml_init = np.array([10, 0.7, 0.4])                #Parameters of gaussian ML [ml0, delta, lower]
beta_init = beta0                                 #Anisotropy parameters 
inc_init = np.array([85])                         #Inclination in deg
log_mbh_init = np.array([8])                      #Log mass of SMBH
qDM_init = np.array([0.5])                        #Scalar describing the axial ratio of DM component
log_rho_s_init = np.array([8])                    #Log intensity of pseudo-NFW profile

##Here we append all the variables in asingle array.
p0 = np.append(ml_init, beta_init)
p0 = np.append(p0,[inc_init, log_mbh_init, qDM_init, log_rho_s_init])

"""
  We will start all walkers in a large gaussian ball around the values above. 
  There is no reason for that, beside the fact that we expect a smooth behaviour of the parameters.  
"""
p0_std = np.abs(p0*0.5)                                  #0.5 is the sigma of the Gaussian ball. 
                                                                #We are using 50% of the initial guesses


#Finally we initialize the walkers with a gaussian ball around the best Collet's fit.
nwalkers = 200                                                  #Number of walkers
pos = emcee.utils.sample_ball(p0, p0_std, nwalkers)             #Initial position of all walkers


nwalkers, ndim = pos.shape


In [7]:
#Backup
filename = "Jam_Emcee_15Parameters.h5"
backend = emcee.backends.HDFBackend(filename)
backend.reset(nwalkers, ndim)

In [8]:
"""
    We save the results in two tables. 
    The first marks the number of iterations, the mean acceptance fraction an the running time. 
    The second marks the last fit values for each parameter.
"""
np.savetxt('Output_LogFile.txt', np.column_stack([0, 0, 0]),
                            fmt=b'	%i	 %e			 %e	 ', 
                            header="Output table for the combined model: Dynamic.\n Iteration	 Mean acceptance fraction	 Processing Time")

np.savetxt("LastFit.txt", np.column_stack([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 
fmt=b'%e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	', 
header="Iteration	 ML0	 Delta	 Lower	 b1	 b2	 b3	 b4	 b5	 b6	 b7	 b8	 Inc	  LogMBH	 qDM	 Logrho_s")

In [9]:
with Pool(4) as pool:
    #Print the number os cores/workers
    print("Workers in this job:", pool._processes)
    print("Start")

    #Initialize the sampler
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, pool=pool, backend=backend)
    
    nsteps = 50000

     # We'll track how the average autocorrelation time estimate changes
    index = 0
    autocorr = np.empty(nsteps)

     # This will be useful to testing convergence
    old_tau = np.inf

    # Now we'll sample for up to max_n steps
    start = time.time()
    global_time = time.time()
    for sample in sampler.sample(pos, iterations=nsteps, progress=True):
        # Only check convergence every 100 steps
        if sampler.iteration % 100:
            continue
        print("\n")
        print("##########################")
        # Compute the autocorrelation time so far
        # Using tol=0 means that we'll always get an estimate even
        # if it isn't trustworthy
        tau = sampler.get_autocorr_time(tol=0)
        autocorr[index] = np.mean(tau)
        index += 1

        #Update a table output with acceptance
        table = np.loadtxt("Output_LogFile.txt")

        iteration = sampler.iteration
        accept = np.mean(sampler.acceptance_fraction)
        total_time = time.time() - global_time
        upt = np.column_stack([iteration, accept, total_time])

        np.savetxt('Output_LogFile.txt', np.vstack([table, upt]),
                                fmt=b'	%i	 %e			 %e	 ', 
                                header="Iteration	 Mean acceptance fraction	 Processing Time")

        #Update table output with last best fit
        last_fit_table = np.loadtxt("LastFit.txt")
        flat_samples = sampler.get_chain(flat=True)
        values = []
        for i in range(ndim):
            mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
            q = np.diff(mcmc)
            values.append(mcmc[1])

        values = np.array(values)
        upt = np.append(iteration, values)

        np.savetxt("LastFit.txt",np.vstack([last_fit_table, upt]),
        fmt=b'%e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	 %e	', 
        header="Iteration	 ML0	 Delta	 Lower	 b1	 b2	 b3	 b4	 b5	 b6	 b7	 b8	 Inc	  LogMBH	 qDM	 Logrho_s")
 

        # Check convergence
        converged = np.all(tau * 100 < sampler.iteration)
        converged &= np.all(np.abs(old_tau - tau) / tau < 0.01)
        if converged:
            break
        old_tau = tau



    end = time.time()
    print('\n')
    print("Final")
    multi_time = end - start
    print("Multiprocessing took {0:.1f} seconds".format(multi_time))

Workers in this job: 4
Start


  0%|          | 0/50000 [00:00<?, ?it/s]/home/carlos/anaconda3/lib/python3.7/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in double_scalars
  lnpdiff = f + nlp - state.log_prob[j]
  0%|          | 5/50000 [00:46<130:11:57,  9.38s/it]

emcee: Exception while calling your likelihood function:

Process ForkPoolWorker-3:
Process ForkPoolWorker-1:


emcee: Exception while calling your likelihood function:

KeyboardInterrupt: 